In [1]:
import pandas as pd
df = pd.read_csv('household_power_consumption.txt', sep=';',  parse_dates={'date_time' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='date_time')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [3]:
df.isnull().values.any()

True

In [4]:
for j in range(0,7):        
        df.iloc[:,j]=df.iloc[:,j].fillna(df.iloc[:,j].mean())

In [5]:
df.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [6]:
# import matplotlib.pyplot as plt
# df.Global_active_power.resample('D').sum().plot(title='Global_active_power resampled over day for sum') 
# #df.Global_active_power.resample('D').mean().plot(title='Global_active_power resampled over day', color='red') 
# plt.tight_layout()
# plt.show()   

In [7]:
print(df[['Global_active_power', 'Global_reactive_power']].corr(method='pearson'))

                       Global_active_power  Global_reactive_power
Global_active_power               1.000000               0.247017
Global_reactive_power             0.247017               1.000000


In [8]:
from scipy import stats
 
correlate = stats.pearsonr(df['Global_active_power'], df['Global_reactive_power'])[0]
correlate

0.24701705264425566

In [9]:
from scipy import stats
 
correlate = stats.pearsonr(df['Voltage'], df['Global_intensity'])[0]
correlate

-0.4113630717685062

In [10]:
df_daily = df.resample('D').mean()
df1 = df_daily[[ 'Global_active_power']]
df1.head()

x_test = df1.last('365D')

x_train = df1 [:1077]

In [11]:
x_train.reset_index(level=0, inplace=True)


In [12]:
x_test.reset_index(level=0, inplace=True)

In [17]:
x_train = x_train.rename(columns={"date_time": "ds", "Global_active_power": "y"})


In [14]:
from fbprophet import Prophet


In [21]:
x_train.reset_index(level=0, inplace=True)
x_test =x_test.rename(columns={"date_time": "ds", "Global_active_power": "y"})



In [23]:
model = Prophet()
model.fit(x_train)

forecast = model.predict(x_test) 
forecast.head()
#yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries




INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2009-11-27,1.096464,1.074250,1.870167,1.096464,1.096464,0.366965,0.366965,0.366965,-0.046197,-0.046197,-0.046197,0.413162,0.413162,0.413162,0.0,0.0,0.0,1.463429
1,2009-11-28,1.096540,1.303821,2.075647,1.096540,1.096540,0.593999,0.593999,0.593999,0.182696,0.182696,0.182696,0.411303,0.411303,0.411303,0.0,0.0,0.0,1.690539
2,2009-11-29,1.096617,1.276540,2.049206,1.096617,1.096617,0.564647,0.564647,0.564647,0.156443,0.156443,0.156443,0.408205,0.408205,0.408205,0.0,0.0,0.0,1.661264
3,2009-11-30,1.096693,1.003557,1.756438,1.096693,1.096693,0.289924,0.289924,0.289924,-0.114240,-0.114240,-0.114240,0.404164,0.404164,0.404164,0.0,0.0,0.0,1.386616
4,2009-12-01,1.096769,1.073162,1.868349,1.096769,1.096769,0.377060,0.377060,0.377060,-0.022428,-0.022428,-0.022428,0.399487,0.399487,0.399487,0.0,0.0,0.0,1.473829


In [27]:
import numpy as np
y_true = x_test['y']
y_pred = forecast['trend']
np.mean(np.abs((y_true - y_pred) / y_true)) * 100

31.82840544558516

In [22]:
x_test

,ds,y
0,2009-11-27,0.958351
1,2009-11-28,1.290937
2,2009-11-29,1.146501
3,2009-11-30,1.211937
4,2009-12-01,1.219707
...,...,...
360,2010-11-22,1.417733
361,2010-11-23,1.095511
362,2010-11-24,1.247394
363,2010-11-25,0.993864


In [26]:
x_test['y']

0      0.958351
1      1.290937
2      1.146501
3      1.211937
4      1.219707
         ...   
360    1.417733
361    1.095511
362    1.247394
363    0.993864
364    1.178230
Name: y, Length: 365, dtype: float64